In [2]:
import pandas as pd
from zipfile36 import ZipFile
import pyarrow as pa
import pyarrow.parquet as pq

In [30]:
zip_file_path = '/workspaces/arcos_all_washpost.zip'
csv_file_name = 'arcos_all_washpost.tsv'
chunk_size = 25000
keep_cols = ['BUYER_COUNTY', 'DOSAGE_UNIT', 'CALC_BASE_WT_IN_GM', 'BUYER_STATE', 'TRANSACTION_DATE', 'DRUG_NAME']
filtered_data = []

with ZipFile(zip_file_path, 'r') as zip_file:
    if csv_file_name in zip_file.namelist():
        with zip_file.open(csv_file_name) as csv_file:
            csv_reader = pd.read_csv(csv_file, delimiter='\t', chunksize=chunk_size, usecols=keep_cols, low_memory=False)
            for i, chunk in enumerate(csv_reader):
                mod_chunk = chunk.loc[(chunk['BUYER_STATE'] == "FL") 
                                      | (chunk['BUYER_STATE'] == "TX") 
                                      | (chunk['BUYER_STATE'] == "WA")]
                mod_chunk['TRANSACTION_DATE'] = mod_chunk['TRANSACTION_DATE'].astype(str)
                try:
                    # Split the date column and convert to integers
                    mod_chunk[['TransactionYear', 'TransactionMonth', 'TransactionDay']] = mod_chunk['TRANSACTION_DATE'].str.split('-', expand=True).astype(int)
                except Exception as e:
                    # Handle the error by inserting NaN values for all three date columns
                    mod_chunk['TransactionYear'] = mod_chunk['TransactionMonth'] = mod_chunk['TransactionDay'] = pd.NA

                # Drop the original 'TRANSACTION_DATE' column
                mod_chunk = mod_chunk.drop(['TRANSACTION_DATE','TransactionDay'], axis=1)
                append_chunk = mod_chunk[(mod_chunk['TransactionYear'] >= 2003) & (mod_chunk['TransactionYear'] <= 2015)]
                #print(f"Processing chunk {i}")
                filtered_data.append(append_chunk)

        selected_data = pd.concat(filtered_data, ignore_index=True)
    else:
        print(f"{csv_file_name} not found in the ZIP file.")

# Specify the path where you want to save the Parquet file
parquet_file_path = 'opioid_transactions.parquet'

# Convert the Pandas DataFrame to a PyArrow Table
table = pa.Table.from_pandas(selected_data)

# Write the Table to a Parquet file
pq.write_table(table, parquet_file_path)

/tmp/ipykernel_23488/3760705209.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_chunk['TRANSACTION_DATE'] = mod_chunk['TRANSACTION_DATE'].astype(str)
/tmp/ipykernel_23488/3760705209.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_chunk[['TransactionYear', 'TransactionMonth', 'TransactionDay']] = mod_chunk['TRANSACTION_DATE'].str.split('-', expand=True).astype(int)
/tmp/ipykernel_23488/3760705209.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [33]:
# Specify the path to the Parquet file you want to open
parquet_file_path = 'opioid_transactions.parquet'

# Open the Parquet file
table = pq.read_table(parquet_file_path)

# Convert the Parquet table to a Pandas DataFrame
df = table.to_pandas()

# Now you can work with the Pandas DataFrame 'df'
# For example, you can print the first few rows
print(df['TransactionYear'].value_counts())

TransactionYear
2011    4383548
2010    4315199
2012    4175389
2013    4147811
2009    4113624
2008    3975294
2007    3809920
2006    3473179
2014    3369605
2015    2770687
Name: count, dtype: int64
